In [2]:
%cd ../

d:\mlops\mlflow_practice


In [3]:
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import mlflow
import numpy as np

In [4]:
with open('assets/feature_store/pca_data.json', 'r') as f:
    combined = json.load(f)
    
metadata = combined['metadata']
print(metadata["description"])
data = combined['data']


df = pd.DataFrame(data)
df.head()

this is pca data from raw


,0,1,2,target
0,-2.264703,0.480027,0.127706,0
1,-2.080961,-0.674134,0.234609,0
2,-2.364229,-0.341908,-0.044201,0
3,-2.299384,-0.597395,-0.091290,0
4,-2.389842,0.646835,-0.015738,0


In [5]:
x= df.drop(columns=["target"])
y= df["target"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [7]:
X_train.shape

(105, 3)

In [8]:
from flowerClassification.config import config
import flowerClassification.utils as utils

{'prep': {'pca_comps_count': 3}, 'logistic_regression': {'C': 0.5}, 'desition_tree': {'max_depth': 3}}


In [9]:
current_branch = utils.get_current_branch()
current_branch

head_commit = utils.get_current_commit_id()
head_commit

'c43ea1f053ddb6b21d195c6429b4fa818b2e5b9f'

In [10]:
config

Box({'prep': {'pca_comps_count': 3}, 'logistic_regression': {'C': 0.5}, 'desition_tree': {'max_depth': 3}})

In [11]:
# Initialize the models
log_reg = LogisticRegression(C = config.logistic_regression.C)
dec_tree = DecisionTreeClassifier(max_depth = config.desition_tree.max_depth )

# Predict and evaluate the models
models = {
    'Logistic Regression': log_reg,
    'Decision Tree': dec_tree
}

In [12]:
mlflow.set_experiment("flowerClassification")


scores={
    
}

for model_name, model in models.items():
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    accuracy = report['accuracy']
    f1_score = report['weighted avg']["f1-score"]
    
    scores={
        **scores,
        f"{model_name}_accuracy":accuracy,
        f"{model_name}_f1_score":f1_score
    }
    
    print(report)    
    print(f"Model: {model_name}")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("-" * 60)



{'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 19.0}, '1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 13.0}, '2': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 13.0}, 'accuracy': 1.0, 'macro avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 45.0}, 'weighted avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 45.0}}
Model: Logistic Regression
Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

------------------------------------------------------------
{'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 19.0}, '1': {'prec

In [13]:
params = {
    "pca_comps" : config.prep.pca_comps_count,
    "lr_C":config.logistic_regression.C,
    "desition_tree_max_depth":config.desition_tree.max_depth
}

In [14]:
scores

{'Logistic Regression_accuracy': 1.0,
 'Logistic Regression_f1_score': 1.0,
 'Decision Tree_accuracy': 0.9333333333333333,
 'Decision Tree_f1_score': 0.9332345679012346}

In [15]:
current_branch,head_commit

('main', 'c43ea1f053ddb6b21d195c6429b4fa818b2e5b9f')

In [16]:
tags={"branch":current_branch , 'commit_id':head_commit}

In [17]:
with open('artifacts/pipelines_scores.json', 'w') as json_file:
    json.dump(scores, json_file, indent=4)
    
with open('artifacts/pipelines_params.json', 'w') as json_file:
    json.dump(params, json_file, indent=4)
    
with open('artifacts/pipelines_info.json', 'w') as json_file:
    json.dump(tags, json_file, indent=4)

In [18]:
with mlflow.start_run(run_name="koko",tags=tags) as run:
    mlflow.log_params(params)
    mlflow.log_metrics(scores)
    mlflow.log_artifacts(local_dir="artifacts")